In [1]:
import pandas as pd

In [2]:
# Reading CSVs and Making DataFrames
reviews = pd.read_csv("data/reviews.csv")
listings = pd.read_csv("data/listings.csv")

In [3]:
reviews["id"].nunique()

133644

In [10]:
# Head
listings.head()
# Number of unique elements for each column
listings.nunique()
# Seeing the host info
listings.iloc[:, 9:26].nunique()
# Value counts of neighbourhoods
listings["neighbourhood_cleansed"].value_counts()


listings["has_availability"].value_counts()

listings[["last_scraped", "calendar_last_scraped"]]

listings["last_scraped"].equals(listings["calendar_last_scraped"])


True